In [25]:
import string
import joblib
from os import path

import numpy as np
import pandas as pd
from scipy import sparse
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm

from utils import ls, cat

tqdm.pandas()

In [26]:
DATA_DIR = '../../data'
OUTPUT_DIR = '../../data'

ls(DATA_DIR)

DIR      ../../data/stopwords-ru
  4.1 MB ../../data/consultant-news-01.01.2022-08.10.2022.csv
  2.7 MB ../../data/consultant-news_lemmas.csv
 38.3 MB ../../data/lenta-economics-01.01.2022-08.10.2022.csv
 24.1 MB ../../data/lenta-economics_lemmas.csv
  2.1 GB ../../data/lenta-ru-news.csv
 20.0  B ../../data/roles.csv
  6.7 MB ../../data/tg-channels.csv
 44.9 MB ../../data/train_data.csv
 21.4 MB ../../data/train_entities.csv
 28.3 MB ../../data/train_lemmas.csv
 22.7 MB ../../data/train_tokens.csv


In [27]:
df_telegram = pd.read_csv('../../data/tg-channels.csv', index_col=0)
df_telegram.head(3)

,url,title,text,topic,tags,date,offset
id,,,,,,,
0,https://t.me/netipichniy_buh/5,NaN,[​​](https://telegra.ph/file/8774505ca2814d361...,NaN,NaN,2020-10-27 03:22:00+00:00,5
1,https://t.me/netipichniy_buh/6,NaN,[​​](https://telegra.ph/file/491be14a0d6a0b2bf...,NaN,NaN,2020-10-27 04:30:01+00:00,6
2,https://t.me/netipichniy_buh/7,NaN,[​​](https://telegra.ph/file/251504c37c3620343...,NaN,NaN,2020-10-30 06:00:06+00:00,7


In [28]:
df_lenta = pd.read_csv('../../data/lenta-economics-01.01.2022-08.10.2022.csv', index_col=0)
df_lenta.head(3)

,url,title,text,topic,tags,date
id,,,,,,
0,https://lenta.ru/news/2022/01/01/bezrbvur/,В России выросли пособия по безработице,В России с 1 января 2022 года вступило в силу ...,Экономика,NaN,2022-01-01 00:10:00
1,https://lenta.ru/news/2022/01/01/mrotvuros/,В России вырос МРОТ,В России вступил в силу федеральный закон № ...,Экономика,NaN,2022-01-01 00:12:00
2,https://lenta.ru/news/2022/01/01/raz/,В России разрешили покупать лекарства онлайн,С 1 января начался эксперимент по онлайн-прода...,Экономика,NaN,2022-01-01 00:23:00


In [29]:
df_consultant = pd.read_csv('../../data/consultant-news-01.01.2022-08.10.2022.csv', index_col=0)
df_consultant.head(3)

,url,title,text,topic,tags,date
id,,,,,,
0,https://www.consultant.ru/legalnews/18344/,"Минтруд утвердил формы, по которым нужно предс...",\n По утвержденным формам работодатели должн...,NaN,NaN,2022-01-31
1,https://www.consultant.ru/legalnews/18343/,В Санкт-Петербурге с 2 февраля нельзя допускат...,\n С 2 по 13 февраля в городе запретили допус...,NaN,NaN,2022-01-31
2,https://www.consultant.ru/legalnews/18342/,Перенос начала применения нормы ФСБУ 27/2021 о...,\n Начало применения абз. 1 п. 25 стандарта ...,NaN,NaN,2022-01-31


In [30]:
columns = ['url', 'title', 'text', 'topic', 'tags', 'date']

df_raw = pd.concat((df_telegram[columns], df_lenta[columns], df_consultant[columns]), axis=0)
df_raw = df_raw.sort_values(by='date').reset_index(drop=True).rename_axis(index='id')
df_raw

,url,title,text,topic,tags,date
id,,,,,,
0,https://t.me/accwhisper/6,NaN,❓**Зачем этот канал? \n**В Сети куча бухгалтер...,NaN,NaN,2018-11-03 19:25:38+00:00
1,https://t.me/accwhisper/9,NaN,А это я в образе переводчика с собачьего. Тьфу...,NaN,NaN,2018-11-03 19:28:15+00:00
2,https://t.me/accwhisper/11,❗️Миф №3. Ваш бухгалтер печется о ваших интере...,**❗️Миф №3. Ваш бухгалтер печется о ваших инте...,NaN,NaN,2018-11-04 17:29:21+00:00
3,https://t.me/accwhisper/10,NaN,👓 **МИФЫ О БУХУЧЕТЕ\n\n**Если подойти к россий...,NaN,NaN,2018-11-04 17:29:21+00:00
4,https://t.me/accwhisper/12,Разрушаем миф №1: Бухучет для налоговой? Как б...,**Разрушаем миф №1: Бухучет для налоговой? Как...,NaN,NaN,2018-11-07 08:44:18+00:00
...,...,...,...,...,...,...
17256,https://www.consultant.ru/legalnews/20478/,Разработали формат заказа или заявки при перев...,\n ФНС выставила на общественное обсуждение пр...,NaN,NaN,2022-10-30
17257,https://www.consultant.ru/legalnews/20425/,Организация не подтвердила дебиторскую задолже...,\n По итогам выездной проверки инспекция приме...,NaN,NaN,2022-10-30
17258,https://www.consultant.ru/legalnews/20477/,Суд: выплату работнику учреждения нельзя призн...,\n Директор образовательного учреждения подал ...,NaN,NaN,2022-10-30


In [31]:
df_train = df_raw[df_raw['date'] > '2022-01-01']
df_train

,url,title,text,topic,tags,date
id,,,,,,
1293,https://lenta.ru/news/2022/01/01/bezrbvur/,В России выросли пособия по безработице,В России с 1 января 2022 года вступило в силу ...,Экономика,NaN,2022-01-01 00:10:00
1294,https://lenta.ru/news/2022/01/01/mrotvuros/,В России вырос МРОТ,В России вступил в силу федеральный закон № ...,Экономика,NaN,2022-01-01 00:12:00
1295,https://lenta.ru/news/2022/01/01/raz/,В России разрешили покупать лекарства онлайн,С 1 января начался эксперимент по онлайн-прода...,Экономика,NaN,2022-01-01 00:23:00
1296,https://lenta.ru/news/2022/01/01/ot/,В России частично отменен один из главных налогов,С 1 января в России от уплаты налога на добавл...,Экономика,NaN,2022-01-01 00:27:00
1297,https://lenta.ru/news/2022/01/01/taba/,В России выросли цены на табак,С 1 января в России минимальная цена на табачн...,Экономика,NaN,2022-01-01 00:38:00
...,...,...,...,...,...,...
17256,https://www.consultant.ru/legalnews/20478/,Разработали формат заказа или заявки при перев...,\n ФНС выставила на общественное обсуждение пр...,NaN,NaN,2022-10-30
17257,https://www.consultant.ru/legalnews/20425/,Организация не подтвердила дебиторскую задолже...,\n По итогам выездной проверки инспекция приме...,NaN,NaN,2022-10-30
17258,https://www.consultant.ru/legalnews/20477/,Суд: выплату работнику учреждения нельзя призн...,\n Директор образовательного учреждения подал ...,NaN,NaN,2022-10-30


In [32]:
df_train.to_csv(f'{DATA_DIR}/train_data.csv')

cat(f'{DATA_DIR}/train_data.csv', 2)

id,url,title,text,topic,tags,date
1293,https://lenta.ru/news/2022/01/01/bezrbvur/,В России выросли пособия по безработице,"В России с 1 января 2022 года вступило в силу  постановление  правительства «О размерах минимальной и максимальной величины пособия по безработице на 2022 год». Выплаты выросли, однако они, как и прежде, соответствуют МРОТ — минимальному показателю оплаты труда за 2021 год. Минимальная величина пособия по безработице осталась на уровне 1,5 тысячи рублей, как и в прошлом году. Максимальная величина в первые три месяца выплаты составила 12 792 рубля (против 12 130 рублей в 2021-м). Выплаты в следующие три месяца остались на уровне в 5000 рублей. Как  говорила  вице-премьер  Татьяна Голикова , в 34 регионах России безработица не вернулась к уровню до пандемии, а в 51 субъекте достигли или превысили докризисные показатели. Замглавы правительства подчеркнула, что официально зарегистрированных безработных в стране стало меньше на 76,2 процента — то есть на 2,8 миллиона ч

## Features

In [33]:
text_cols = ['title', 'text']

df_train[text_cols].head()

,title,text
id,,
1293,В России выросли пособия по безработице,В России с 1 января 2022 года вступило в силу ...
1294,В России вырос МРОТ,В России вступил в силу федеральный закон № ...
1295,В России разрешили покупать лекарства онлайн,С 1 января начался эксперимент по онлайн-прода...
1296,В России частично отменен один из главных налогов,С 1 января в России от уплаты налога на добавл...
1297,В России выросли цены на табак,С 1 января в России минимальная цена на табачн...


In [34]:
df_train[text_cols].fillna('').applymap(len).describe()

,title,text
count,15968.000000,15968.000000
mean,56.334857,1445.934557
std,20.347896,809.622589
min,0.000000,1.000000
25%,47.000000,992.750000
50%,58.000000,1260.000000
75%,69.000000,1673.000000
max,134.000000,18905.000000


Хорошо, у большей части новостей даже заполнен заголовок!

## Lemmatized Text

In [35]:
# !pip install spacy
# python -m spacy download ru_core_news_md

In [36]:
import spacy

nlp = spacy.load('ru_core_news_md', disable=['tagger', 'attribute_ruler', 'senter', 'parser', 'ner'])

In [37]:
train_tokens = df_train[text_cols].fillna('') \
    .progress_applymap(nlp)
    
train_tokens

100%|█████████████████████████████████████| 31936/31936 [10:57<00:00, 48.61it/s]


,title,text
id,,
1293,"(В, России, выросли, пособия, по, безработице)","(В, России, с, 1, января, 2022, года, вступило..."
1294,"(В, России, вырос, МРОТ)","(В, России, вступил, в, силу, федеральный, , ..."
1295,"(В, России, разрешили, покупать, лекарства, он...","(С, 1, января, начался, эксперимент, по, онлай..."
1296,"(В, России, частично, отменен, один, из, главн...","(С, 1, января, в, России, от, уплаты, налога, ..."
1297,"(В, России, выросли, цены, на, табак)","(С, 1, января, в, России, минимальная, цена, н..."
...,...,...
17256,"(Разработали, формат, заказа, или, заявки, при...","(\n , ФНС, выставила, на, общественное, обсужд..."
17257,"(Организация, не, подтвердила, дебиторскую, за...","(\n , По, итогам, выездной, проверки, инспекци..."
17258,"(Суд, :, выплату, работнику, учреждения, нельз...","(\n , Директор, образовательного, учреждения, ..."


In [39]:
TRAIN_LEMMAS_PATH = f'{OUTPUT_DIR}/train_lemmas_with_date.csv'


def is_alphanum(token):
    return not token.is_punct \
        and not token.is_currency \
        and not token.is_digit \
        and not token.is_punct \
        and not token.is_oov \
        and not token.is_space \
        and not token.is_stop \
        and not token.like_num \
        and not token.pos_ == "PROPN"

df_lemmas = train_tokens \
    .progress_applymap(lambda x: ' '.join(t.lemma_ for t in x if is_alphanum(t)))

df_lemmas['date'] = df_train['date']
df_lemmas.to_csv(TRAIN_LEMMAS_PATH)

ls(OUTPUT_DIR, path.basename(TRAIN_LEMMAS_PATH))

100%|██████████████████████████████████| 31936/31936 [00:03<00:00, 10612.23it/s]


 28.6 MB ../../data/train_lemmas_with_date.csv


In [40]:
df_lemmas

,title,text,date
id,,,
1293,вырасти пособие безработица,январь год вступить сила постановление правите...,2022-01-01 00:10:00
1294,вырасти,вступить сила федеральный закон внесение измен...,2022-01-01 00:12:00
1295,разрешить покупать лекарство онлайн,январь начаться эксперимент онлайн продаже лек...,2022-01-01 00:23:00
1296,частично отменить главный налог,январь уплата налог добавить стоимость освобод...,2022-01-01 00:27:00
1297,вырасти цена табак,январь минимальный цена табачный продукция дос...,2022-01-01 00:38:00
...,...,...,...
17256,формат заказ заявка перевозка автомобиль,выставить общественный обсуждение проект форма...,2022-10-30
17257,подтвердить задолженность мера законный решить...,итог выездной проверка инспекция применить мер...,2022-10-30
17258,выплата работник учреждение признать оформление,образовательный учреждение подать учредитель з...,2022-10-30


## Lemmatization and verb removing

In [42]:
TRAIN_ENTITIES_PATH = f'{OUTPUT_DIR}/train_entities_with_date.csv'


def is_alphanum_not_verb(token):
    return not token.is_punct \
        and not token.is_currency \
        and not token.is_digit \
        and not token.is_punct \
        and not token.is_oov \
        and not token.is_space \
        and not token.is_stop \
        and not token.like_num \
        and not token.pos_ == 'PROPN' \
        and not token.pos_ == 'VERB'

df_entities = train_tokens \
    .progress_applymap(lambda x: ' '.join(t.lemma_ for t in x if is_alphanum_not_verb(t)))

df_entities['date'] = df_train['date']
df_entities.to_csv(TRAIN_ENTITIES_PATH)

ls(OUTPUT_DIR, path.basename(TRAIN_ENTITIES_PATH))

100%|██████████████████████████████████| 31936/31936 [00:02<00:00, 10696.48it/s]


 21.7 MB ../../data/train_entities_with_date.csv


In [43]:
df_entities

,title,text,date
id,,,
1293,пособие безработица,январь год сила постановление правительство ра...,2022-01-01 00:10:00
1294,,сила федеральный закон внесение изменение стат...,2022-01-01 00:12:00
1295,лекарство онлайн,январь эксперимент онлайн продаже лекарство са...,2022-01-01 00:23:00
1296,частично главный налог,январь уплата налог стоимость компания сфера о...,2022-01-01 00:27:00
1297,цена табак,январь минимальный цена табачный продукция руб...,2022-01-01 00:38:00
...,...,...,...
17256,формат заказ заявка перевозка автомобиль,общественный обсуждение проект формат заказ за...,2022-10-30
17257,задолженность мера законный суд,итог выездной проверка инспекция мера имуществ...,2022-10-30
17258,выплата работник учреждение оформление,образовательный учреждение учредитель заявлени...,2022-10-30
